In [42]:
import dask.dataframe as dd
import numpy as np
import pandas as pd
import ray
from basefunc import genbasis_df, get_test_data_df, index_csum
from genericray import *
from sklearn.linear_model import BayesianRidge

In [43]:
# Initialize Ray
context = ray.init(ignore_reinit_error=True)
print(context.dashboard_url)

2024-08-07 11:13:07,216	INFO worker.py:1614 -- Calling ray.init() again after it has already been called.


127.0.0.1:8265


In [44]:
chunk_size = 36
num_basis_vectors = 5
dt = chunk_size / num_basis_vectors

# Generate test data
delta_df, original_df = get_test_data_df()


delta_df = pd.Series(np.random.standard_normal(size=1_000_000))

rolling_vol = delta_df.rolling(60).std()
rolling_vol[rolling_vol == 0] = np.nan
rolling_vol = rolling_vol.ffill().bfill()
rolling_vol_l1 = rolling_vol.shift(1).ffill().bfill()
delta_df /= rolling_vol_l1

cumsum_df = index_csum(delta_df, chunk_size)

/home/jthor/projects/tsprojection/src/bpr


In [45]:
basis_df = genbasis_df(dt, chunk_size, num_basis_vectors)
basis_df = (
    basis_df.compute() if isinstance(basis_df, dd.DataFrame) else basis_df
)

In [46]:
weights_df_generic = apply_model(
    dd.from_pandas(cumsum_df, npartitions=7), basis_df, chunk_size
)
print("Generic Application Weights:")
print(weights_df_generic)

Generic Application Weights:
               0         1         2         3         4
35      0.152236 -0.070147 -0.501474  0.209293  0.050561
71      0.287405 -0.280182 -0.271397  0.781765 -1.168640
107    -0.094912 -0.073123  0.100903  0.027370 -0.080301
143    -0.932527  0.911936  0.163777 -0.068189  0.200876
179     0.100012  0.101007 -0.252092 -0.091990 -0.102264
...          ...       ...       ...       ...       ...
999846 -0.394788  0.235485  0.023319  0.328765  0.119113
999882 -0.131027 -0.172991  0.144491 -0.535222  0.866933
999918 -0.339328  0.046053  1.607446 -1.605514  0.253081
999954  0.180628  0.110710  0.017378  0.150566 -0.074726
999990 -0.152861 -0.187443 -1.224462  1.729183  0.035845

[27776 rows x 5 columns]


: 